In [1]:
!pip install nemo_toolkit['all']

# TTS-EN-FASTPITCH

In [3]:
!pip install pynini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 MB 4.1 MB/s eta 0:00:00


tts_en_fastpitch

In [4]:
# Load FastPitch
from nemo.collections.tts.models import FastPitchModel
spec_generator = FastPitchModel.from_pretrained("nvidia/tts_en_fastpitch")

[NeMo I 2023-01-06 11:19:07 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2023-01-06 11:19:57 experimental:27] Module <class 'nemo_text_processing.g2p.modules.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-01-06 11:19:59 modules:95] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2023-01-06 11:19:59 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_m

[NeMo I 2023-01-06 11:20:00 features:267] PADDING: 1
[NeMo I 2023-01-06 11:20:01 save_restore_connector:243] Model FastPitchModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--tts_en_fastpitch/snapshots/2c8305b7b41b33fd6367f0635796dc3a7a33cbf9/tts_en_fastpitch.nemo.


In [5]:
# Load vocoder
from nemo.collections.tts.models import HifiGanModel
model = HifiGanModel.from_pretrained(model_name="nvidia/tts_hifigan")

Downloading:   0%|          | 0.00/315M [00:00<?, ?B/s]

[NeMo W 2023-01-06 11:20:19 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2023-01-06 11:20:19 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2023-01-06 11:20:19 features:267] PADDING: 0


[NeMo W 2023-01-06 11:20:19 features:244] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2023-01-06 11:20:19 features:267] PADDING: 0
[NeMo I 2023-01-06 11:20:21 save_restore_connector:243] Model HifiGanModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--tts_hifigan/snapshots/3ba1fed954276287015654bf4c78060ffc9a4772/tts_hifigan.nemo.


In [6]:
import soundfile as sf
parsed = spec_generator.parse("""You don’t have to worry about retraining your model or how to publish the new model. 
Your model is updated on a regular basis. Furthermore, 
if you decide to retrain your model for whatever reason, it will not be an issue; 
you can just update the model on the cloud, and everyone will benefit.""")
spectrogram = spec_generator.generate_spectrogram(tokens=parsed)
audio = model.convert_spectrogram_to_audio(spec=spectrogram)

[NeMo W 2023-01-06 11:20:33 fastpitch:260] parse() is meant to be called in eval mode.
[NeMo W 2023-01-06 11:20:33 fastpitch:325] generate_spectrogram() is meant to be called in eval mode.


In [7]:
# Save the audio to disk in a file called speech.wav
sf.write("tts_en_fastpitch.wav", audio.to('cpu').detach().numpy()[0], 22050) ##human alike

TTS-EN-TACOTRON2

In [8]:
import soundfile as sf
import nemo
from nemo.collections.tts.models.base import SpectrogramGenerator, Vocoder

In [ ]:
# Download and load the pretrained tacotron2 model
spec_generator = SpectrogramGenerator.from_pretrained("tts_en_tacotron2")

# Download and load the pretrained waveglow model
vocoder = Vocoder.from_pretrained("tts_waveglow_88m")

In [10]:
# All spectrogram generators start by parsing raw strings to a tokenized version of the string
parsed = spec_generator.parse("""You don’t have to worry about retraining your model or how to publish the new model. 
Your model is updated on a regular basis. Furthermore, 
if you decide to retrain your model for whatever reason, it will not be an issue; 
you can just update the model on the cloud, and everyone will benefit.""")
# Then take the tokenized string and produce a spectrogram
spectrogram = spec_generator.generate_spectrogram(tokens=parsed)
# Finally, a vocoder converts the spectrogram to audio
audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)

[NeMo W 2023-01-06 11:23:42 tacotron2:145] parse() is meant to be called in eval mode.
[NeMo W 2023-01-06 11:23:53 tacotron2:341] Reached max decoder steps 1000.
[NeMo W 2023-01-06 11:25:59 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
      return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
    


In [11]:
# Save the audio to disk in a file called speech.wav
sf.write("tts-waveglow-88m.wav", audio.to('cpu').detach().numpy()[0], 22050)############very bad

tts_en_fastspeech2



In [12]:
# Load FastSpeech 2
from nemo.collections.tts.models import FastSpeech2Model
spec_generator = FastSpeech2Model.from_pretrained("tts_en_fastspeech2")

ImportError: ignored

In [13]:

# Load vocoder
from nemo.collections.tts.models.base import Vocoder
model = Vocoder.from_pretrained(model_name="tts_hifigan")

[NeMo I 2023-01-06 11:29:19 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_hifigan/versions/1.0.0rc1/files/tts_hifigan.nemo to /root/.cache/torch/NeMo/NeMo_1.14.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2023-01-06 11:29:26 cloud:79] Download from cloud failed. Attempt 1 of 3
[NeMo I 2023-01-06 11:29:36 common:912] Instantiating model from pre-trained checkpoint


[NeMo W 2023-01-06 11:29:40 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2023-01-06 11:29:40 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2023-01-06 11:29:40 features:267] PADDING: 0


[NeMo W 2023-01-06 11:29:40 features:244] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2023-01-06 11:29:40 features:267] PADDING: 0
[NeMo I 2023-01-06 11:29:42 save_restore_connector:243] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.14.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [ ]:
# Generate audio
import soundfile as sf
parsed = spec_generator.parse("""You don’t have to worry about retraining your model or how to publish the new model. 
Your model is updated on a regular basis. Furthermore, 
if you decide to retrain your model for whatever reason, it will not be an issue; 
you can just update the model on the cloud, and everyone will benefit.""")
spectrogram = spec_generator.generate_spectrogram(tokens=parsed)
audio = model.convert_spectrogram_to_audio(spec=spectrogram)

# Save the audio to disk in a file called speech.wav
sf.write("tts_en_fastspeech2.wav", audio.to('cpu').detach().numpy()[0], 22050)

MIXER-TTS-X

In [14]:
# Load Mixer-TTS-X
from nemo.collections.tts.models import MixerTTSModel
spec_generator = MixerTTSModel.from_pretrained("tts_en_lj_mixerttsx")

[NeMo I 2023-01-06 11:29:42 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_en_lj_mixerttsx/versions/1.6.0/files/tts_en_lj_mixerttsx.nemo to /root/.cache/torch/NeMo/NeMo_1.14.0/tts_en_lj_mixerttsx/3f77e9a4e457b181ac7f332517093fc0/tts_en_lj_mixerttsx.nemo
[NeMo I 2023-01-06 11:29:47 common:912] Instantiating model from pre-trained checkpoint
[NeMo I 2023-01-06 11:29:50 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2023-01-06 11:30:28 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.MixerTTSDataset
      manifest_filepath: lj_train.json
      sample_rate: 22050
      sup_data_path: sup_data_mixer_tts_dataset_align_prior_matrix_pitch_mixer_tts_231221
      sup_data_types:
      - align_prior_matrix
      - pitch
      - lm_tokens
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_duration: 0.1
      ignore_file: null
      trim: false
      pitch_fmin: 65.40639132514966
      pitch_fmax: 2093.004522404789
      lm_model: albert
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 32
      num_workers: 4
      pin_memory: false
   

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.decoder.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[NeMo I 2023-01-06 11:30:31 features:267] PADDING: 1
[NeMo I 2023-01-06 11:30:31 save_restore_connector:243] Model MixerTTSModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.14.0/tts_en_lj_mixerttsx/3f77e9a4e457b181ac7f332517093fc0/tts_en_lj_mixerttsx.nemo.


In [15]:
# Load vocoder
from nemo.collections.tts.models.base import SpectrogramGenerator, Vocoder
model = Vocoder.from_pretrained(model_name="tts_en_lj_hifigan_ft_mixerttsx")

[NeMo I 2023-01-06 11:30:31 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_en_lj_hifigan/versions/1.6.0/files/tts_en_lj_hifigan_ft_mixerttsx.nemo to /root/.cache/torch/NeMo/NeMo_1.14.0/tts_en_lj_hifigan_ft_mixerttsx/2286838f886ea3e82d0c3348b67e5035/tts_en_lj_hifigan_ft_mixerttsx.nemo
[NeMo I 2023-01-06 11:30:44 common:912] Instantiating model from pre-trained checkpoint


[NeMo W 2023-01-06 11:30:50 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: lj_train.json
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 16
      num_workers: 1
    
[NeMo W 2023-01-06 11:30:50 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: lj_val.json
      min_duration: 3
      n_segments: 66048
    dataloader_params:
      drop_last: false


[NeMo I 2023-01-06 11:30:50 features:267] PADDING: 0
[NeMo I 2023-01-06 11:30:50 features:275] STFT using exact pad
[NeMo I 2023-01-06 11:30:50 features:267] PADDING: 0
[NeMo I 2023-01-06 11:30:50 features:275] STFT using exact pad
[NeMo I 2023-01-06 11:30:51 save_restore_connector:243] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.14.0/tts_en_lj_hifigan_ft_mixerttsx/2286838f886ea3e82d0c3348b67e5035/tts_en_lj_hifigan_ft_mixerttsx.nemo.


In [18]:

# Generate audio
import soundfile as sf
raw_text = """You don’t have to worry about retraining your model or how to publish the new model. Your 
model is updated on a regular basis. Furthermore, if you decide to retrain your model for whatever reason, 
it will not be an issue; you can just update the model on the cloud, and everyone will benefit."""
parsed = spec_generator.parse(raw_text)
spectrogram = spec_generator.generate_spectrogram(tokens=parsed, raw_texts=[raw_text])
audio = model.convert_spectrogram_to_audio(spec=spectrogram)

[NeMo W 2023-01-06 11:36:46 mixer_tts:658] parse() is meant to be called in eval mode.


In [19]:
# Save the audio to disk in a file called speech.wav
sf.write("mixer-tts-en.wav", audio.to('cpu').numpy(), 22050)

RuntimeError: ignored

GlowTTS

In [ ]:
# Load GlowTTS
from nemo.collections.tts.models import GlowTTSModel
spec_generator = GlowTTSModel.from_pretrained("tts_en_glowtts")

# Load vocoder
from nemo.collections.tts.models import Vocoder
model = Vocoder.from_pretrained(model_name="tts_hifigan")

# Generate audio
import soundfile as sf
parsed = spec_generator.parse("You can type your sentence here to get nemo to produce speech.")
spectrogram = spec_generator.generate_spectrogram(tokens=parsed)
audio = model.convert_spectrogram_to_audio(spec=spectrogram)

# Save the audio to disk in a file called speech.wav
sf.write("speech.wav", audio.to('cpu').numpy(), 22050)

tts_en_libritts_multispeaker_univnet

In [ ]:
# Load PastPitch
from nemo.collections.tts.models import FastPitchModel
spec_generator = FastPitchModel.from_pretrained("tts_en_fastpitch")


In [ ]:
# Load UnivNet
from nemo.collections.tts.models import Vocoder
model = Vocoder.from_pretrained(model_name="tts_en_libritts_multispeaker_univnet")

In [ ]:
# Generate audio
import soundfile as sf
parsed = spec_generator.parse("""Rationality means that an AI agent is assumed to take 
account of available information and uncertainty, potential costs 
and benefits, and to act consistently (logically) in choosing the best action""")
spectrogram = spec_generator.generate_spectrogram(tokens=parsed)
audio = model.convert_spectrogram_to_audio(spec=spectrogram)

### Save the audio to disk in a file called speech.wav
sf.write("tts_en_libritts_multispeaker_univnet.wav", audio.to('cpu').detach().numpy()[0], 22050)

tts_en_e2e_fastspeech2hifigan

In [ ]:
import soundfile as sf
from nemo.collections.tts.models.

# Load the model from NGC
model = HifiGanModel.from_pretrained(model_name="tts_en_e2e_fastspeech2hifigan")

In [ ]:
# Run inference
tokens = model.parse("Hey, I can speak!")
audio = model.convert_text_to_waveform(tokens=tokens)

# Save the audio to disk in a file called speech.wav
sf.write("speech.wav", audio.to('cpu').numpy(), 22050)

tts_en_e2e_fastpitchhifigan

In [ ]:
import soundfile as sf
from nemo.collections.tts.models import FastPitchHifiGanE2EModel

# Load the model from NGC
model = FastPitchHifiGanE2EModel.from_pretrained(model_name="tts_en_e2e_fastpitchhifigan")

# Run inference
tokens = model.parse("Hey, I can speak!")
audio = model.convert_text_to_waveform(tokens=tokens)

# Save the audio to disk in a file called speech.wav
sf.write("tts_en_e2e_fastpitchhifigan.wav", audio.to('cpu').numpy(), 22050)

**VITS**

In [22]:
!pip install espnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 KB 20.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 KB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
from espnet2.bin.tts_inference import Text2Speech

model = Text2Speech.from_pretrained("espnet/kan-bayashi_ljspeech_vits")

speech, *_ = model("""You don’t have to worry about retraining your model or how to publish the new model.
Your model is updated on a regular basis. Furthermore, if you decide to retrain your model for whatever reason,
 it will not be an issue; you can just update the model on the cloud, and everyone will benefit.""")

ModuleNotFoundError: ignored

Refer below links:
https://catalog.ngc.nvidia.com/orgs/nvidia/collections/nemo_tts
https://youtu.be/1Bmg1c5U5Bg
https://catalog.ngc.nvidia.com/orgs/nvidia/resources/flowtron/quick-start-guide
https://paperswithcode.com/sota/text-to-speech-synthesis-on-ljspeech
https://catalog.ngc.nvidia.com/orgs/nvidia/teams/adlr/models/flowtron

